# To Contracept or Not to Contracept

Import packeges and py-files

In [1]:
# a. set magic 
%load_ext autoreload
%autoreload 2

# b. packeges 
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import copy

# c. py-files 
from Model import child_model 
import Solve as solver
import Estimate as estimate

## Set up

### Read data

In [5]:
model = child_model()
dta = model.read_data()

### State trantision conditional on age and choice from the real data

In [6]:
solver.P_list(model, dta)

## Show that the estimating works on simulated data

### Solve the model and simulate data

In [7]:
# a. Solve model 
V, pnc = solver.BackwardsInduction(model)

# b. Simulate data
data = model.sim_data(pnc)

### Chosen parameters

In [ ]:
print(model.mu, model.eta1, model.eta2)

### Estimate parameters from simulated data

In [9]:
# a. Initialize parameters 
samplesize = data.shape[0]
par_mu = copy.copy(model.mu)
par_eta1 = copy.copy(model.eta1)
par_eta2 = copy.copy(model.eta2)

# b. Number of gridpoints (must be the same size)
I = 50

# c. Search grid 
log_lik = np.nan + np.zeros((I,I,I))
mu = np.linspace(0,1.8,I)
eta1 = np.linspace(0,1.8,I)
eta2 = np.linspace(-0.9,0.9,I)

# d. Loop over all parameter grids 
for i in range(I):
    for j in range(I):
        for k in range(I):
                # i. Estimate parameters
                pnames = ['eta1', 'eta2', 'mu']
                theta = np.array([eta1[i], eta2[j], mu[k]])
                # ii. Estimate log-likelihood in simulated data
                log_lik[i,j,k] = estimate.ll(theta, model, solver, data, pnames)

# e. Log-likelihood 
log_lik *= samplesize*(-1)

# f. Re-inset the true parameters 
model.eta1 = copy.copy(par_eta1)
model.eta2 = copy.copy(par_eta2)
model.mu = copy.copy(par_mu)

# g. Find the maximum log-likelihood
x, y, z = np.unravel_index(np.argmax(log_lik), log_lik.shape)

# h. Print results
print(eta1[x], eta2[y], mu[z])

KeyboardInterrupt: 

## Estimate parameters on real-life data

In [ ]:
# a. Initialize parameters 
samplesize = dta.shape[0]
par_mu = copy.copy(model.mu)
par_eta1 = copy.copy(model.eta1)
par_eta2 = copy.copy(model.eta2)

# b. Number of gridpoints
I = 50

# c. Search grid 
log_lik = np.nan + np.zeros((I,I,I))
mu = np.linspace(0,1.8,I)
eta1 = np.linspace(0,1.8,I)
eta2 = np.linspace(-0.9,0.9,I)

# d. Loop over all parameter grids 
for i in range(I):
    for j in range(I):
        for k in range(I):
                
                # i. Estimate parameters
                pnames = ['eta1', 'eta2', 'mu']
                theta = np.array([eta1[i], eta2[j], mu[k]])
                # ii. Estimate log-likelihood in real data
                log_lik[i,j,k] = estimate.ll(theta, model, solver, dta, pnames)

# e. Log-likelihood 
log_lik *= samplesize*(-1)

# f. Re-inset the true parameters 
model.eta1 = copy.copy(par_eta1)
model.eta2 = copy.copy(par_eta2)
model.mu = copy.copy(par_mu)

# g. Find the maximum log-likelihood
x, y, z = np.unravel_index(np.argmax(log_lik), log_lik.shape)

# h. Print results
print(eta1[x], eta2[y], mu[z])

### Update parameters

In [ ]:
model.mu = 
model.eta1 = 
model.eta2 =

In [ ]:
#løs model og simuler data med rigtige parametre
V, pnc = solver.BackwardsInduction(model)
data = model.sim_data(pnc)

In [ ]:
#kig på fordelingen af børn i sidste periode
data0 = data[(data['t']==model.T-1)]
data0.x.value_counts()/model.N

In [ ]:
# Kig på choice probability
print(pnc)

## Counter factuals

### Perfect contraceptive use

In [ ]:
# a. Perfect contraceptive use 
model.p2_list = np.ones([model.T,2])*np.array([1, 0])

# b. Solve model
V_cf, pnc_cf = solver.BackwardsInduction(model)

# c. Simulate data
data_cf = model.sim_data(pnc_cf)

### Distrubution of children

In [ ]:
# a. Data in last period
data_cf0 = data_cf[(data_cf['t']==model.T-1)]

# b. Count number of children
data_cf0.x.value_counts()/model.N

### Choice probabilities

In [ ]:
print(pnc_cf)